# Locating the references to the MicroSats in email
- found in GMAIL with this search "`chaz.hyseni@gmail.com micro update`"
- he included sequence for `D101` but only a pretty useless link to a paper with only primers listed for `Gmm8`
- `Gmm8` sequence located via google at http://goo.gl/W1nL8g

# Seq file created

### MicroSat data:
```
FILENAME: /home/gus/remote_mounts/louise/data/projects/ddrad58/seqs/chaz_environment_microsats.fasta

>D101 with flanking regions
GCATATATTGTGAAAAGCTCATGCGTCTGCCTTTACACTGCATACTACCAGCTATAATTGGTTGTTGCAAGAGCTGAGTCTTATTGGCTGATTGGTTGCTCATACTATATGCGCGCTCTGGTTGATTAGAGAAAACTACTGTTGCTGCTGCTGCTGCTGCTGCTGCCGCTGCTGCTATCCAACACATCATTGCTCCTCTTTTTTTTTATTGTGCATTTTTGTTTAGCAGAATATTCGCTCTTAACCTACGGTTTGTGGATAGTAATAAGCAAAAGAGAAGAAGAAAAAGAAAAAGAAGAAGAAGAAAAAGCAAAAGCAGAATATGCGATTTTTATTCTCCACATAATATGTTTATGCCTTCTCCGTTGCAATAAGATTTTCTCGTTTTTGTTGTTTTTCGAGTGTTTCATCATCTGTTGCCAATTGTCAGTTGCAACGCGTC

>Gmm8 from morsitans (http://goo.gl/W1nL8g)
CATATGACTGAACATTATATCATGCAGATGCAATGCGGAGAGAGAGAGAGAGAGAGAGAGTGAGAGAGAGTGAGATTGAA
AGCAGCAGACTGCACGTAGGGTCGACTGGGCGTCATTGTCAGTTGCGAAAGCAAACATTGAAGCGCGCCCAAGCAAATGA
ACGAGCAACTGAGCAAACAGACAAACAAATAAACAAACATTACAGCTAAACATTAAATACATAGAAAGT
```



# BLASTing Vectorbase
